<a href="https://colab.research.google.com/github/team-10-ACM-ML/week-3/blob/master/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1.112486e+09
1,1,29,3.5,1.112485e+09
2,1,32,3.5,1.112485e+09
3,1,47,3.5,1.112485e+09
4,1,50,3.5,1.112485e+09


In [4]:
tags = pd.read_csv('tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [5]:
links = pd.read_csv('links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
a = ratings['userId'].value_counts()

In [7]:
b = ratings['userId'].value_counts()>100

In [8]:
new_users=list(a[b].index)

In [9]:
len(new_users)

44414

In [10]:
ratings=ratings[ratings['userId'].isin(new_users)]

In [11]:
ratings.shape

(13669867, 4)

In [12]:
ratings_with_moviename=ratings.merge(movies,on='movieId')

In [13]:
ratings_with_moviename.shape

(13669867, 6)

In [14]:
ratings_with_moviename.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1.112486e+09,Jumanji (1995),Adventure|Children|Fantasy
1,29,2,3.0,8.355622e+08,Jumanji (1995),Adventure|Children|Fantasy
2,54,2,3.0,9.749182e+08,Jumanji (1995),Adventure|Children|Fantasy
3,88,2,1.0,1.098278e+09,Jumanji (1995),Adventure|Children|Fantasy
4,91,2,3.5,1.112061e+09,Jumanji (1995),Adventure|Children|Fantasy


In [15]:
movie_num_ratings=ratings_with_moviename.groupby('title')['rating'].count().reset_index()

In [16]:
movie_num_ratings.rename(columns={'rating': 'num_ratings'},inplace=True)

In [17]:
movie_num_ratings=movie_num_ratings[movie_num_ratings['num_ratings']>=50]

In [18]:
movie_num_ratings

,title,num_ratings
0,"""Great Performances"" Cats (1998)",50
11,'Round Midnight (1986),119
12,'Salem's Lot (2004),68
13,'Til There Was You (1997),350
15,"'burbs, The (1989)",2268
...,...,...
25510,loudQUIETloud: A Film About the Pixies (2006),81
25511,xXx (2002),3545
25512,xXx: State of the Union (2005),584
25515,¡Three Amigos! (1986),4698


In [19]:
final_rating=ratings_with_moviename.merge(movie_num_ratings, on='title')

In [20]:
final_rating.shape

(13522606, 7)

In [21]:
final_rating.head()

,userId,movieId,rating,timestamp,title,genres,num_ratings
0,1,2,3.5,1.112486e+09,Jumanji (1995),Adventure|Children|Fantasy,13912
1,29,2,3.0,8.355622e+08,Jumanji (1995),Adventure|Children|Fantasy,13912
2,54,2,3.0,9.749182e+08,Jumanji (1995),Adventure|Children|Fantasy,13912
3,88,2,1.0,1.098278e+09,Jumanji (1995),Adventure|Children|Fantasy,13912
4,91,2,3.5,1.112061e+09,Jumanji (1995),Adventure|Children|Fantasy,13912


In [22]:
final_rating=final_rating.drop_duplicates(subset=['userId','title'])

In [23]:
final_rating.shape

(13522339, 7)

In [24]:
final_rating

,userId,movieId,rating,timestamp,title,genres,num_ratings
0,1,2,3.5,1.112486e+09,Jumanji (1995),Adventure|Children|Fantasy,13912
1,29,2,3.0,8.355622e+08,Jumanji (1995),Adventure|Children|Fantasy,13912
2,54,2,3.0,9.749182e+08,Jumanji (1995),Adventure|Children|Fantasy,13912
3,88,2,1.0,1.098278e+09,Jumanji (1995),Adventure|Children|Fantasy,13912
4,91,2,3.5,1.112061e+09,Jumanji (1995),Adventure|Children|Fantasy,13912
...,...,...,...,...,...,...,...
13522601,109210,26463,3.5,1.289589e+09,Barefoot Gen (Hadashi no Gen) (1983),Animation|Drama|War,53
13522602,109737,26463,4.0,1.284756e+09,Barefoot Gen (Hadashi no Gen) (1983),Animation|Drama|War,53
13522603,112365,26463,2.0,1.356645e+09,Barefoot Gen (Hadashi no Gen) (1983),Animation|Drama|War,53
13522604,114406,26463,2.0,1.290914e+09,Barefoot Gen (Hadashi no Gen) (1983),Animation|Drama|War,53


In [25]:
movie_pivot=final_rating.pivot_table(columns='userId',index='title',values='rating')

In [26]:
movie_pivot.shape

(9867, 44414)

In [27]:
movie_pivot=movie_pivot.fillna(0)

In [28]:
movie_pivot

userId,1,3,7,11,14,18,21,22,23,24,25,29,31,35,46,51,53,54,56,58,61,69,70,73,82,88,89,90,91,96,101,103,104,109,112,116,124,129,130,131,...,118221,118222,118226,118229,118235,118236,118238,118239,118243,118247,118249,118252,118253,118254,118255,118256,118258,118260,118265,118266,118268,118269,118270,118271,118272,118273,118275,118277,118279,118281,118285,118288,118289,118291,118296,118298,118303,118310,118311,118313
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
loudQUIETloud: A Film About the Pixies (2006),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xXx (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0
xXx: State of the Union (2005),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
from scipy.sparse import csr_matrix

In [30]:
movie_matrix=csr_matrix(movie_pivot)

In [31]:
from sklearn.neighbors import NearestNeighbors

In [32]:
model=NearestNeighbors(algorithm='brute')

In [33]:
model.fit(movie_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [34]:
movie_pivot.index[200]

'Addams Family Reunion (1998)'

In [35]:
movie_pivot.iloc[200,:].values.reshape(1,-1)

array([[0., 0., 0., ..., 0., 0., 0.]])

In [36]:
distance,suggestion=model.kneighbors(movie_pivot.iloc[1,:].values.reshape(1,-1),n_neighbors=5)

In [37]:
suggestion

array([[   1, 4120,  314, 8486,  851]])

In [39]:
movie_pivot.index[102]

'35 Up (1991)'

In [40]:
for i in suggestion[0]:
  print(movie_pivot.index[i])

'Round Midnight (1986)
Horrors of Spider Island (Ein Toter Hing im Netz) (1960)
Alien from L.A. (1988)
SuperBabies: Baby Geniuses 2 (2004)
Beast of Yucca Flats, The (1961)


In [41]:
def recommend(moviename):
    m_id=np.where(movie_pivot.index==moviename)[0][0]
    distance,suggestion=model.kneighbors(movie_pivot.iloc[m_id,:].values.reshape(1,-1),n_neighbors=5)
    for i in suggestion[0]:
        print(movie_pivot.index[i])

In [46]:
recommend('Alien from L.A. (1988)')

Alien from L.A. (1988)
Beast of Yucca Flats, The (1961)
Horrors of Spider Island (Ein Toter Hing im Netz) (1960)
Mitchell (1975)
SuperBabies: Baby Geniuses 2 (2004)


In [48]:
recommend('Jumanji (1995)')

Jumanji (1995)
Casper (1995)
Santa Clause, The (1994)
Flintstones, The (1994)
Ace Ventura: When Nature Calls (1995)
